In [1]:
#!pip3 install missing packages

import numpy as np
import re
import requests
import pandas as pd
import seaborn as sns
from functools import reduce

from igel import Igel

import matplotlib.pyplot as plt
from matplotlib import rcParams

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Helper Functions

In [2]:
def clean_dataset(df):
    #assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    #df.dropna(inplace=True)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df
    #return df[indices_to_keep].astype(np.float64)
    

def clean_dataset_int(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df[indices_to_keep].astype(np.float64)

## Mappings

Defining various mappings we'll use when manipulating the data.

In [3]:
priority_scorer = {
    '10' : 'Critical',
    '9' : 'Critical',
    '8' : 'Critical',
    '7' : 'High',
    '6' : 'High',
    '5' : 'Medium',
    '4' : 'Medium',
    '3' : 'Low',
    '2' : 'Low',
    '1.0' : 'Low',
    '0.0' : 'Low',
}

priority_mapping = {
    "Critical" : 10,
    "High" : 7.5,
    "Medium" : 5,
    "Low" : 2.5,
    "Unknown" : 0,
}

# What we consider to be highly important categories of information
highCategoriser = {
    0.0 : 'Other-Advice',
    1.0 : 'Other-Advice',
    2.0 : 'Report-CleanUp',
    3.0 : 'ContextualInformation',
    4.0 : 'Other-ContextualInformation',
    5.0 : 'CallToAction-Donations',
    6.0 : 'Report-EmergingThreats',
    7.0 : 'Report-Factoid',
    8.0 : 'Report-FirstPartyObservation',
    9.0 : 'Request-GoodsServices',
    10.0 : 'Report-Hashtags',
    11.0 : 'Request-InformationWanted',
    12.0 : 'Other-Irrelevant',
    13.0 : 'Report-Location',
    14.0 : 'CallToAction-MovePeople',
    15.0 : 'Report-MultimediaShare',
    16.0 : 'Report-NewSubEvent',
    17.0 : 'Report-News',
    18.0 : 'Report-Official',
    19.0 : 'Report-OriginalEvent',
    20.0 : 'Request-SearchAndRescue',
    21.0 : 'Other-Sentiment',
    22.0 : 'Report-ServiceAvailable',
    23.0 : 'Report-ThirdPartyObservation',
    24.0 : 'CallToAction-Volunteer',
    25.0 : 'Report-Weather',
    26.0 : 'hmm',
}

event_int_map =	{
  "guatemalaEarthquake2012": 7,
  "joplinTornado2011": 16,
  "athensEarthquake2020": 35,
  "baltimoreFlashFlood2020": 36,
  "brooklynBlockPartyShooting2020": 37,
  "daytonOhioShooting2020": 38,
  "elPasoWalmartShooting2020": 39,
  "gilroygarlicShooting2020": 40,
  "hurricaneBarry2020": 41,
  "indonesiaEarthquake2020": 42,
  "keralaFloods2020": 43,
  "myanmarFloods2020": 44,
  "papuaNewguineaEarthquake2020": 45,
  "siberianWildfires2020": 46,
  "typhoonKrosa2020": 47,
  "typhoonLekima2020": 48,
  "whaleyBridgeCollapse2020": 49
}

mymap = {'Advice':1, 'CleanUp':2, 'ContextualInformation':3, 'Discussion':4, 'Donations':5, 
        'EmergingThreats':6, 'Factoid':7, 'FirstPartyObservation':8, 'GoodsServices':9, 'Hashtags':10, 
        'InformationWanted':11,'Irrelevant':12, 'Location':13, 'MovePeople':14, 
         'MultimediaShare':15, 'NewSubEvent':16, 'News':17,
        'Official':18, 'OriginalEvent':19, 'SearchAndRescue':20, 'Sentiment':21, 'ServiceAvailable':22, 
         'ThirdPartyObservation':23,'Volunteer':24, 'Weather':25}

## FeatureAPI

Load the feature vector in from Play

In [4]:
#url = 'http://localhost:9000/stored_tweets'
import json
def parse_json_stream(stream):
    decoder = json.JSONDecoder()
    while stream:
        obj, idx = decoder.raw_decode(stream)
        yield obj
        stream = stream[idx:].lstrip()

#url = 'http://tweetminer-2336003gproject.ida.dcs.gla.ac.uk/stored_tweets'

#data = requests.get(url)#.json()

#result = json.loads(data.content.decode('utf-8'))

#q = requests.get(url)

#json_response = json.loads(q.text.replace('�', ''))



In [5]:
# JSONDecodeError: Expecting ',' delimiter: 
#json_response = json.loads('[' + q.text + '],')

#print(data)

In [6]:
file_list = ["../output/run.json", "../output/run0.json"] #"../output/run.json", ,"../output/run0.json"
dfs = [] # an empty list to store the data frames

for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
    dfs.append(json_data)
df = pd.concat(dfs, sort=False) # or sort=True depending on your needs
df




,fileName,tweets
0,data/tweets/run/brook_selected.jsonl,"[{'tweet_id': 1155531927056551936, 'tweet_text..."
1,data/tweets/run/el.jsonl,"[{'tweet_id': 1157817871835324416, 'tweet_text..."
2,data/tweets/run/gil.jsonl,"[{'tweet_id': 1155854599221329920, 'tweet_text..."
0,data/tweets/run0/indo.jsonl,"[{'tweet_id': 1157394630561808384, 'tweet_text..."
1,data/tweets/run0/indo.jsonl,"[{'tweet_id': 1157394630561808384, 'tweet_text..."
2,data/tweets/run0/barry.jsonl,"[{'tweet_id': 1149871010231070720, 'tweet_text..."
3,data/tweets/run0/barry.jsonl,"[{'tweet_id': 1149871010231070720, 'tweet_text..."
4,data/tweets/run0/dayton_selected.jsonl,"[{'tweet_id': 1158114685738266627, 'tweet_text..."
5,data/tweets/run0/dayton_selected.jsonl,"[{'tweet_id': 1158114685738266627, 'tweet_text..."
6,data/tweets/run0/baltimore_selected.jsonl,"[{'tweet_id': 1157961073921810432, 'tweet_text..."


In [7]:
#df = pd.read_json("../output/run.json", lines=True)
from pandas.io.json import json_normalize
df = pd.concat([pd.DataFrame(json_normalize(x)) for x in df['tweets']],ignore_index=True)
df

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,tfIdf.son,tfIdf.children,tfIdf.future,tfIdf.ground,tfIdf.charged,tfIdf.fear,tfIdf.narrative,tfIdf.interrupt,tfIdf.standard,tfIdf.london,tfIdf.evening,...,tfIdf.bus,tfIdf.scare,tfIdf.gleybers,tfIdf.translation,tfIdf.awaits,tfIdf.tonights,tfIdf.raven,tfIdf.sunset,tfIdf.loch,tfIdf.reservoir,tfIdf.filled,tfIdf.bills,tfIdf.ransomware,tfIdf.funding,tfIdf.federal,tfIdf.soared,tfIdf.sank,tfIdf.outchea,tfIdf.leoseason,tfIdf.tufff,tfIdf.echoing,tfIdf.chrome,tfIdf.f

In [8]:
#df = pd.DataFrame(new_df)
#df = pd.json_normalize(df['tweets'])

#df.to_csv("temp/df.csv", index=False)


# Normalise the sentiment

In [9]:
df['sentiment'] = df[df['sentiment'].apply(lambda x:pd.Series(x))]


INFO - NumExpr defaulting to 8 threads.


In [10]:
df_sentiment=pd.concat([df['sentiment'].str.split(',', expand=True)], axis=1, keys="s")

In [11]:
# Join back onto the original dataframe
df.join(df_sentiment)

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,tfIdf.son,tfIdf.children,tfIdf.future,tfIdf.ground,tfIdf.charged,tfIdf.fear,tfIdf.narrative,tfIdf.interrupt,tfIdf.standard,tfIdf.london,tfIdf.evening,...,tfIdf.scare,tfIdf.gleybers,tfIdf.translation,tfIdf.awaits,tfIdf.tonights,tfIdf.raven,tfIdf.sunset,tfIdf.loch,tfIdf.reservoir,tfIdf.filled,tfIdf.bills,tfIdf.ransomware,tfIdf.funding,tfIdf.federal,tfIdf.soared,tfIdf.sank,tfIdf.outchea,tfIdf.leoseason,tfIdf.tufff,tfIdf.echoing,tfIdf.chrome,tfIdf.fordtauruss

# Normalise the embeddings

In [12]:
df['embeddings'] = df['embeddings'].str.extract('(\d+)', expand=False)


In [13]:
df['embeddings'] = df['embeddings'].str[1:]

In [14]:
df_embeddings = pd.concat([df['embeddings'].str.split(',', expand=True)], axis=1, keys="e")
df_embeddings

,e
,0
0,
1,
2,
3,
4,
...,...
8047,
8048,
8049,


In [15]:
# Join back onto the original dataframe
df.join(df_embeddings)

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,tfIdf.son,tfIdf.children,tfIdf.future,tfIdf.ground,tfIdf.charged,tfIdf.fear,tfIdf.narrative,tfIdf.interrupt,tfIdf.standard,tfIdf.london,tfIdf.evening,...,tfIdf.scare,tfIdf.gleybers,tfIdf.translation,tfIdf.awaits,tfIdf.tonights,tfIdf.raven,tfIdf.sunset,tfIdf.loch,tfIdf.reservoir,tfIdf.filled,tfIdf.bills,tfIdf.ransomware,tfIdf.funding,tfIdf.federal,tfIdf.soared,tfIdf.sank,tfIdf.outchea,tfIdf.leoseason,tfIdf.tufff,tfIdf.echoing,tfIdf.chrome,tfIdf.fordtauruss

In [16]:
df_emb_sent=df_sentiment.join(df_embeddings)
df_emb_sent


,s,e
,0,0
0,NaN,
1,NaN,
2,NaN,
3,NaN,
4,NaN,
...,...,...
8047,NaN,
8048,NaN,
8049,NaN,


In [17]:
df.join(df_emb_sent)

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,tfIdf.son,tfIdf.children,tfIdf.future,tfIdf.ground,tfIdf.charged,tfIdf.fear,tfIdf.narrative,tfIdf.interrupt,tfIdf.standard,tfIdf.london,tfIdf.evening,...,tfIdf.gleybers,tfIdf.translation,tfIdf.awaits,tfIdf.tonights,tfIdf.raven,tfIdf.sunset,tfIdf.loch,tfIdf.reservoir,tfIdf.filled,tfIdf.bills,tfIdf.ransomware,tfIdf.funding,tfIdf.federal,tfIdf.soared,tfIdf.sank,tfIdf.outchea,tfIdf.leoseason,tfIdf.tufff,tfIdf.echoing,tfIdf.chrome,tfIdf.fordtaurussho,tfIdf.rgf

In [22]:
df

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,tfIdf.son,tfIdf.children,tfIdf.future,tfIdf.ground,tfIdf.charged,tfIdf.fear,tfIdf.narrative,tfIdf.interrupt,tfIdf.standard,tfIdf.london,tfIdf.evening,...,tfIdf.bus,tfIdf.scare,tfIdf.gleybers,tfIdf.translation,tfIdf.awaits,tfIdf.tonights,tfIdf.raven,tfIdf.sunset,tfIdf.loch,tfIdf.reservoir,tfIdf.filled,tfIdf.bills,tfIdf.ransomware,tfIdf.funding,tfIdf.federal,tfIdf.soared,tfIdf.sank,tfIdf.outchea,tfIdf.leoseason,tfIdf.tufff,tfIdf.echoing,tfIdf.chrome,tfIdf.f

In [19]:


# Export
#df.to_csv("temp/api_input3.csv", index=False)

api_input = pd.read_csv("temp/api_input.csv")
api_input.head()



,tweet_id,tweet_text,offset,embeddings,sentiment,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.rattled,tfIdf.residents,tfIdf.earthquake,tfIdf.indonesia,tfIdf.jakarta,tfIdf.tsunami,tfIdf.java,tfIdf.powerful,tfIdf.quake,tfIdf.pakistan,tfIdf.warning,tfIdf.lifts,tfIdf.strong,tfIdf.sumatra,tfIdf.southwest,tfIdf.coast,tfIdf.hit,tfIdf.shallow,tfIdf.occurred,tfIdf.utc,tfIdf.southwestofsumatra,tfIdf.fecha,tfIdf.temblor,tfIdf.sismo,tfIdf.prof,tfIdf.gmt,tfIdf.depth,tfIdf.epicenter,tfIdf.geohack,tfIdf.magnitude,tfIdf.region,tfIdf.coords,tfIdf.info,tfIdf.banten,tfIdf.lampung,tfIdf.bengkulu,tfIdf.west,tfIdf.issued,tfIdf.breaking,tfIdf.struck,tfIdf.indonesias,tfIdf.news,tfIdf.low,tfIdf.lalaupdates,tfIdf.weather,tfIdf.tomorrow,tfIdf.sunny,tfIdf.issuing,tfIdf.indonesian,tfIdf.read,tfIdf.living,tfIdf.authorities,tfIdf.urged,tfIdf.wake,tfIdf.islands,tfIdf.friday,tfIdf.ground,tfIdf.hits,tfIdf.evacuate,tfIdf.buildings,tfIdf.afp,tfIdf.indonesians,tfIdf.earthquakes,tfIdf.island,tfIdf.strikes,tfIdf.cuties,tfIdf.safe,tfIdf.happened,tfIdf.stay,tfIdf.haskell,tfIdf.senior,tfIdf.derivatives,tfIdf.london,tfIdf.tdd,tfIdf.alert,tfIdf.rate,tfIdf.developer,tfIdf.pricing,tfIdf.banking,tfIdf.gempa,tfIdf.hope,tfIdf.likrly,tfIdf.jakbar,tfIdf.tsunamis,tfIdf.worst,tfIdf.bye,tfIdf.fears,tfIdf.missed,tfIdf.active,tfIdf.country,tfIdf.cities,tfIdf.rocks,tfIdf.live,tfIdf.panic,tfIdf.kills,tfIdf.displaced,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.dead,tfIdf.capital,tfIdf.swaying,tfIdf.cancelled,tfIdf.nstworld,tfIdf.populated,tfIdf.heavily,tfIdf.died,tfIdf.triggering,tfIdf.injured,tfIdf.rocked,tfIdf.usgs,tfIdf.nav,tfIdf.faircom,tfIdf.learn,tfIdf.developers,tfIdf.dbms,tfIdf.nodejs,tfIdf.ctreeace,tfIdf.demand,tfIdf.sql,tfIdf.nosql,tfIdf.database,tfIdf.performance,tfIdf.meet,tfIdf.api,tfIdf.embedded,tfIdf.designed,tfIdf.released,tfIdf.feel,tfIdf.peeps,tfIdf.ago,tfIdf.scared,tfIdf.yup,tfIdf.spectral,tfIdf.dec,tfIdf.dynamicearthcommunity,tfIdf.representation,tfIdf.damaged,tfIdf.officials,tfIdf.houses,tfIdf.huge,tfIdf.allah,tfIdf.lah,tfIdf.month,tfIdf.ya,tfIdf.dah,tfIdf.constantly,tfIdf.kilometres,tfIdf.loves,tfIdf.nuest,tfIdf.manually,tfIdf.mw,tfIdf.ball,tfIdf.beach,tfIdf.revised,tfIdf.status,tfIdf.wamnews,tfIdf.mitigation,tfIdf.agency,tfIdf.disaster,tfIdf.fine,tfIdf.nyoongs,tfIdf.mn,tfIdf.netflix,tfIdf.sso,tfIdf.spring,tfIdf.hour,tfIdf.scrum,tfIdf.boot,tfIdf.oauth,tfIdf.minneapolis,tfIdf.fuck,tfIdf.scaring,tfIdf.shakes,tfIdf.travel,tfIdf.national,tfIdf.development,tfIdf.python,tfIdf.android,tfIdf.golang,tfIdf.learning,tfIdf.datascience,tfIdf.ios,tfIdf.artificialintelligence,tfIdf.reactjs,tfIdf.gt,tfIdf.javascript,tfIdf.network,tfIdf.ruby,tfIdf.coding,tfIdf.angular,tfIdf.security,tfIdf.csharp,tfIdf.linux,tfIdf.ionic,tfIdf.vuejs,tfIdf.perl,tfIdf.programming,tfIdf.orang,tfIdf.safety,tfIdf.bed,tfIdf.finger,tfIdf.shook,tfIdf.cat,tfIdf.eaten,tfIdf.yall,tfIdf.boiz,tfIdf.expecting,tfIdf.motorcycle,tfIdf.law,tfIdf.lives,tfIdf.triplets,tfIdf.wife,tfIdf.brother,tfIdf.south,...,tfIdf.eastsyde,tfIdf.wanna,tfIdf.psa,tfIdf.waits,tfIdf.peep,tfIdf.libs,tfIdf.muniland,tfIdf.hoping,tfIdf.truth,tfIdf.horrors,tfIdf.comparatively,tfIdf.shit,tfIdf.gronk,tfIdf.aaron,tfIdf.combo,tfIdf.hernandez,tfIdf.orange,tfIdf.metro,tfIdf.severe,tfIdf.thunderstorm,tfIdf.conservativestakecharge,tfIdf

In [20]:
api_input3 = pd.read_csv("temp/api_input3.csv")
api_input3.head()

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,tfIdf.son,tfIdf.children,tfIdf.future,tfIdf.ground,tfIdf.charged,tfIdf.fear,tfIdf.narrative,tfIdf.interrupt,tfIdf.standard,tfIdf.london,tfIdf.evening,...,tfIdf.wtaf,tfIdf.craxy,tfIdf.causing,tfIdf.instruments,tfIdf.ndamendment,tfIdf.preserving,tfIdf.canadanewsmedia,tfIdf.react,tfIdf.racistpresident,tfIdf.racisminamerica,tfIdf.twoamericas,tfIdf.fb,tfIdf.effected,tfIdf.iblametrump,tfIdf.ted,tfIdf.cruz,tfIdf.likes,tfIdf.confirms,tfIdf.prioritising,tfIdf.prof

In [21]:
new_df = api_input3.join(api_input, on='tweet_id', how='left')

ValueError: columns overlap but no suffix specified: Index(['tweet_id', 'tweet_text', 'offset', 'sentiment', 'embeddings', 'features.numb_of_urls', 'features.numb_of_hashtags', 'features.numb_of_personal_pronouns', 'features.numb_of_present_tenses', 'features.weighted_length',
       ...
       'tfIdf.loss', 'tfIdf.idiotic', 'tfIdf.changing', 'tfIdf.fest', 'tfIdf.salmon', 'tfIdf.arm', 'tfIdf.eat', 'tfIdf.prepare', 'tfIdf.cute', 'tfIdf.shook'], dtype='object', length=931)

In [ ]:
new_df

In [ ]:
api_input = new_df

# Drop the string categories
#api_input.drop(['tweet_text_y'], axis = 1, inplace = True)
#api_input.drop(['embeddings_x'], axis = 1, inplace = True)
#api_input.drop(['sentiment_x'], axis = 1, inplace = True)

api_input.describe()

In [23]:
# Group by tweet
feature_vector_input = df.groupby(['tweet_id']).agg('first')
feature_vector_input

,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,tfIdf.son,tfIdf.children,tfIdf.future,tfIdf.ground,tfIdf.charged,tfIdf.fear,tfIdf.narrative,tfIdf.interrupt,tfIdf.standard,tfIdf.london,tfIdf.evening,tfIdf.nero,...,tfIdf.bus,tfIdf.scare,tfIdf.gleybers,tfIdf.translation,tfIdf.awaits,tfIdf.tonights,tfIdf.raven,tfIdf.sunset,tfIdf.loch,tfIdf.reservoir,tfIdf.filled,tfIdf.bills,tfIdf.ransomware,tfIdf.funding,tfIdf.federal,tfIdf.soared,tfIdf.sank,tfIdf.outchea,tfIdf.leoseason,tfIdf.tufff,tfIdf.echoing,tfIdf.chrome,tfIdf

## Load the labelled data

These are generated in 0_Labels.ipynb

In [24]:
# dtypes needs to be specified or long ints will change
labels_df = pd.read_json("../../../data/input/raw/data/2020/2020-A/labels/TRECIS-2018-2020A-labels.json", dtype={} )

# replace the event with a numeric value
labels_df = labels_df.replace({'eventID': event_int_map})

# Count the number of labels
labels_df['num'] = labels_df['postCategories'].str.len()

# Map the priority to a numeric value
labels_df = labels_df.replace({"postPriority": priority_mapping})

# Split categories and map to numeric values
category_list = pd.DataFrame(labels_df["postCategories"].to_list(), columns=['cat1', 'cat2', 'cat3',
                                                                   'cat4', 'cat5', 'cat6',
                                                                   'cat7', 'cat8', 'cat9', 'cat10'])


# Map the categories to numeric values
category_list = category_list.applymap(lambda s: mymap.get(s) if s in mymap else s)


# Join back onto our original list
labels = labels_df.join(category_list)

# Drop the string categories
labels.drop(['postCategories'], axis = 1, inplace = True)

# Tidy
labels = labels.drop(['eventName', 'eventDescription', 'eventType'], axis=1)

# Fill the NaN slots with 0
labels = labels.fillna("0")

# Export
labels.to_csv("../labels.csv", index=False)
labels = clean_dataset_int(labels)
#labels = clean_dataset_new(labels)

labels

,eventID,postID,postPriority,num,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10
0,16.0,7.267628e+16,2.5,3.0,7.0,10.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16.0,7.267840e+16,10.0,4.0,22.0,18.0,10.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0
2,16.0,7.268240e+16,2.5,2.0,21.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16.0,7.269393e+16,5.0,3.0,23.0,10.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16.0,7.269856e+16,2.5,3.0,23.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42946,47.0,1.162000e+18,2.5,1.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42947,47.0,1.162005e+18,2.5,3.0,13.0,15.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42948,47.0,1.162005e+18,2.5,2.0,13.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42949,47.0,1.162006e+18,2.5,3.0,13.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Train.csv


We merge the feature vector with the annotated data, this is used to train the classifier.

In [36]:
feature_vector_input.drop(['tweet_text'], axis = 1, inplace = True)
feature_vector_input.drop(['embeddings'], axis = 1, inplace = True)
feature_vector_input.drop(['sentiment'], axis = 1, inplace = True)

In [37]:
# Merges the input feature vector with the labels
train = pd.merge(labels, feature_vector_input, left_on = 'postID', right_on = 'tweet_id', how = 'inner')

# reset the index
train = train.reset_index()

# fill NaN and replace Infinity
train.fillna(0, inplace=True)
train.replace([np.inf, -np.inf], np.nan, inplace=True)
train = train.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

# export to csv
train.to_csv("../train.csv", index=False)

train

,index,eventID,postID,postPriority,num,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,...,tfIdf.bus,tfIdf.scare,tfIdf.gleybers,tfIdf.translation,tfIdf.awaits,tfIdf.tonights,tfIdf.raven,tfIdf.sunset,tfIdf.loch,tfIdf.reservoir,tfIdf.filled,tfIdf.bills,tfIdf.ransomware,tfIdf.funding,tfIdf.federal,tfIdf.soared,tfIdf.sank,tfIdf.outchea,tfIdf.leoseason,tfIdf.tufff,tfIdf.echoing,tfIdf.chrome,tfIdf.fordtaurussho,tfIdf.rgfxcustoms,tfIdf.ford,tfIdf.delete,tfIdf.mchromedelete,tfIdf.stealthmode,tfIdf.tau

# Test.csv


Drops the categories, number of categories and priority so we can make our prediction

In [38]:
# Take a copy of the train dataframe
test = train

# Drop cat*
test.drop(list(test.filter(regex = 'cat\d+$')), axis = 1, inplace = True)


# Drop priority / num (of labels)
test.drop(['postPriority', 'num'], axis = 1, inplace = True)

#api_input.drop(['tweet_text_y'], axis = 1, inplace = True)
#api_input.drop(['embeddings_x'], axis = 1, inplace = True)
#api_input.drop(['sentiment_x'], axis = 1, inplace = True)

# export
test.to_csv("../test.csv", index=False)

test

,index,eventID,postID,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,tfIdf.son,tfIdf.children,tfIdf.future,tfIdf.ground,tfIdf.charged,tfIdf.fear,tfIdf.narrative,tfIdf.interrupt,tfIdf.standard,tfIdf.london,tfIdf.evening,tfIdf.nero,...,tfIdf.bus,tfIdf.scare,tfIdf.gleybers,tfIdf.translation,tfIdf.awaits,tfIdf.tonights,tfIdf.raven,tfIdf.sunset,tfIdf.loch,tfIdf.reservoir,tfIdf.filled,tfIdf.bills,tfIdf.ransomware,tfIdf.funding,tfIdf.federal,tfIdf.soared,tfIdf.sank,tfIdf.outchea,tfIdf.leoseason,tfIdf.tufff,tfIdf.echoing,tfIdf.chrome,tfIdf.fordtaurus

# Igel

This notebook implements `Igel` and supports all sklearn's machine learning functionality.

Caution must be taken to avoid overfitting. See `docs/ml.md` for more information

Igel's supported models:

        +--------------------+----------------------------+-------------------------+
        |      regression    |        classification      |        clustering       |
        +--------------------+----------------------------+-------------------------+
        |   LinearRegression |         LogisticRegression |                  KMeans |
        |              Lasso |                      Ridge |     AffinityPropagation |
        |          LassoLars |               DecisionTree |                   Birch |
        | BayesianRegression |                  ExtraTree | AgglomerativeClustering |
        |    HuberRegression |               RandomForest |    FeatureAgglomeration |
        |              Ridge |                 ExtraTrees |                  DBSCAN |
        |  PoissonRegression |                        SVM |         MiniBatchKMeans |
        |      ARDRegression |                  LinearSVM |    SpectralBiclustering |
        |  TweedieRegression |                      NuSVM |    SpectralCoclustering |
        | TheilSenRegression |            NearestNeighbor |      SpectralClustering |
        |    GammaRegression |              NeuralNetwork |               MeanShift |
        |   RANSACRegression | PassiveAgressiveClassifier |                  OPTICS |
        |       DecisionTree |                 Perceptron |                    ---- |
        |          ExtraTree |               BernoulliRBM |                    ---- |
        |       RandomForest |           BoltzmannMachine |                    ---- |
        |         ExtraTrees |       CalibratedClassifier |                    ---- |
        |                SVM |                   Adaboost |                    ---- |
        |          LinearSVM |                    Bagging |                    ---- |
        |              NuSVM |           GradientBoosting |                    ---- |
        |    NearestNeighbor |        BernoulliNaiveBayes |                    ---- |
        |      NeuralNetwork |      CategoricalNaiveBayes |                    ---- |
        |         ElasticNet |       ComplementNaiveBayes |                    ---- |
        |       BernoulliRBM |         GaussianNaiveBayes |                    ---- |
        |   BoltzmannMachine |      MultinomialNaiveBayes |                    ---- |
        |           Adaboost |                       ---- |                    ---- |
        |            Bagging |                       ---- |                    ---- |
        |   GradientBoosting |                       ---- |                    ---- |
        +--------------------+----------------------------+-------------------------+

In [39]:
# Train the model

params = {
        'cmd': 'fit',    
        'data_path': "../train.csv",
        'yaml_path': 'yaml/multi.yaml'  # DecisionTree
}

Igel(**params)

INFO - Entered CLI args: {'cmd': 'fit', 'data_path': '../train.csv', 'yaml_path': 'yaml/multi.yaml'}
INFO - Executing command: fit ...
INFO - reading data from ../train.csv
INFO - You passed the configurations as a yaml file.
INFO - your chosen configuration: {'dataset': {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default', 'scale': {'method': 'standard', 'target': 'inputs'}}}, 'model': {'type': 'regression', 'algorithm': 'DecisionTree'}, 'target': ['postPriority', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'num']}
INFO - dataset_props: {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default', 'scale': {'method': 'standard', 'target': 'inputs'}}} 
model_props: {'type': 'regression', 'algorithm': 'DecisionTree'} 
 target: ['postPriority', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'num'] 

INFO - dataset shape: (1777, 5684)
INFO - dataset attributes: ['index', 'eventID', 'postID', 'post

INFO - y shape: (1777, 12) and x shape: (1777, 5672)
INFO - Solving a regression problem using ===> DecisionTree
INFO - model arguments: 
None
INFO - executing a DecisionTreeRegressor algorithm...
INFO - predicting multiple targets detected. Hence, the model will be automatically converted to a multioutput model
INFO - cross validation is not provided
INFO - Folder /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results already exists
WARNING - data in the /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results folder will be overridden. If you don't want this, then move the current /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results to another path
INFO - Successfully created the directory in /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results 
INFO - model saved successfully and can be found in the /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results folder
INFO - split option detected. The performance will b

In [40]:
# Use model to predict on missing values

params = {
        'cmd': 'predict',    
        'data_path': "../test.csv",
        'yaml_path': 'yaml/hyper.yaml'
}
Igel(**params)

INFO - Entered CLI args: {'cmd': 'predict', 'data_path': '../test.csv', 'yaml_path': 'yaml/hyper.yaml'}
INFO - Executing command: predict ...
INFO - reading data from ../test.csv
INFO - path of the pre-fitted model => /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results/model.sav
INFO - loading from /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results/model.sav
INFO - dataset shape: (1777, 5672)
INFO - dataset attributes: ['index', 'eventID', 'postID', 'offset', 'features.numb_of_urls', 'features.numb_of_hashtags', 'features.numb_of_personal_pronouns', 'features.numb_of_present_tenses', 'features.weighted_length', 'features.permillage', 'features.tweet_created_at', 'features.tweet_id_str', 'features.positive_sentiment', 'features.negative_sentiment', 'features.numb_of_mentions', 'features.numb_of_media', 'features.numb_of_past_tenses', 'features.numb_of_weird_chars', 'features.numb_of_questions', 'features.numb_of_emoticons', 'features.numb_of_swearing_w

INFO - predictions shape: (1777, 12) | shape len: 2
INFO - predict on targets: ['postPriority', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'num']
INFO - saving the predictions to /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results/predictions.csv


# Predictions

1. View the raw predictions
2. Map the labels to their High Level Information Types
3. Merge the predictions back into the training set


In [41]:
def is_neg_predictions(predictions):
    predictions = predictions.sort_values(by=['postPriority'])
    predictions = predictions[(predictions > 0).all(1)]
    predictions.round()

    
predictions = pd.read_csv("model_results/predictions.csv")
predictions

#is_neg_predictions(predictions)

,postPriority,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,num
0,2.5,23.0,13.0,7.0,18.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0
1,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2.5,13.0,7.0,18.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5.0,25.0,13.0,7.0,18.0,19.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1772,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1773,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1774,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1775,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Merge the new predictions back onto dataframe with the missing columns

In [42]:
# Map the labels to their High Level Information Types
cat_list = predictions.filter(regex='cat', axis=1).round().applymap(lambda x: highCategoriser[x])

#
predictions = cat_list.combine_first(predictions)

# Merge the predictions back into the training set
df = test.merge(predictions, left_index=True, right_index=True)


# Append the predicted categories to a list in a new column
df['predicted_categories'] = df[['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10']].values.tolist()


# Get the number of categories into something we can use to index
df['num'] = df['num'].astype(float).astype(int)

# Remove categories beyond what the tweet is predicted to have
df['categories'] = df.apply(lambda x: x['predicted_categories'][0:x['num']], axis=1)


df

,index,eventID,postID,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.brooklyn,tfIdf.shooting,tfIdf.everybodys,tfIdf.social,tfIdf.ahead,tfIdf.brownsville,tfIdf.claimed,tfIdf.media,tfIdf.community,tfIdf.claiming,tfIdf.gunfire,tfIdf.amazing,tfIdf.celebration,tfIdf.crazy,tfIdf.days,tfIdf.course,tfIdf.kids,tfIdf.deadly,tfIdf.southall,tfIdf.ashley,tfIdf.nyt,tfIdf.block,tfIdf.beloved,tfIdf.party,tfIdf.marred,tfIdf.gun,tfIdf.tragedy,tfIdf.low,tfIdf.suffering,tfIdf.forward,tfIdf.nyc,tfIdf.crime,tfIdf.event,tfIdf.yearly,tfIdf.constituents,tfIdf.communities,tfIdf.violence,tfIdf.wounded,tfIdf.left,tfIdf.dead,tfIdf.injured,tfIdf.jerusalem,tfIdf.leaves,tfIdf.nypd,tfIdf.park,tfIdf.shattered,tfIdf.mayor,tfIdf.york,tfIdf.blasio,tfIdf.neighborhood,tfIdf.peaceful,tfIdf.ideal,tfIdf.shooters,tfIdf.water,tfIdf.shot,tfIdf.throwing,tfIdf.police,tfIdf.wonder,tfIdf.officers,tfIdf.provide,tfIdf.person,tfIdf.officials,tfIdf.updates,tfIdf.vows,tfIdf.justice,tfIdf.killed,tfIdf.commissioner,tfIdf.gunman,tfIdf.mass,tfIdf.held,tfIdf.city,tfIdf.scene,tfIdf.attack,tfIdf.rushed,tfIdf.playground,tfIdf.paramedics,tfIdf.forget,tfIdf.night,tfIdf.talking,tfIdf.worse,tfIdf.crowded,tfIdf.seriously,tfIdf.cover,tfIdf.running,tfIdf.outdoor,tfIdf.late,tfIdf.hundreds,tfIdf.thousands,tfIdf.hurt,tfIdf.normal,tfIdf.multiple,tfIdf.stoptheviolence,tfIdf.prayers,tfIdf.damn,tfIdf.minutes,tfIdf.drove,tfIdf.spot,tfIdf.canarsie,tfIdf.happened,tfIdf.news,tfIdf.gunmen,tfIdf.nypds,tfIdf.oneill,tfIdf.todau,tfIdf.catch,tfIdf.didn,tfIdf.wasn,tfIdf.wonderful,tfIdf.talk,tfIdf.leadership,tfIdf.safe,tfIdf.ny,tfIdf.conditions,tfIdf.adverse,tfIdf.hey,tfIdf.oldtimers,tfIdf.week,tfIdf.st,tfIdf.bed,tfIdf.ave,tfIdf.stuy,tfIdf.report,tfIdf.howard,tfIdf.marked,tfIdf.pieces,tfIdf.evidence,tfIdf.condemning,tfIdf.morning,tfIdf.columbus,tfIdf.loose,tfIdf.hospitalized,tfIdf.head,tfIdf.single,tfIdf.timers,tfIdf.wound,tfIdf.erupted,tfIdf.died,tfIdf.bullet,tfIdf.fox,tfIdf.god,tfIdf.tw,tfIdf.amp,tfIdf.htmlsocsrc,tfIdf.soctrk,tfIdf.lives,tfIdf.shooter,tfIdf.devastated,tfIdf.interrupted,tfIdf.family,tfIdf.residence,tfIdf.arrests,tfIdf.update,tfIdf.fighting,tfIdf.gunwoman,tfIdf.stupid,tfIdf.lies,tfIdf.danger,tfIdf.breaking,tfIdf.trumps,tfIdf.fault,tfIdf.maga,tfIdf.president,tfIdf.guess,tfIdf.playing,tfIdf.killing,tfIdf.gathered,tfIdf.injuring,tfIdf.firing,tfIdf.annual,tfIdf.set,tfIdf.weekend,tfIdf.scendsei,tfIdf.snuck,tfIdf.tremendous,tfIdf.summer,tfIdf.believes,tfIdf.peace,tfIdf.silver,tfIdf.blessing,tfIdf.critical,tfIdf.festival,tfIdf.condition,tfIdf.leaving,tfIdf.close,tfIdf.coming,tfIdf.sadly,tfIdf.saturday,tfIdf.represents,tfIdf.democrat,tfIdf.district,tfIdf.bra,tfIdf.cue,tfIdf.music,tfIdf.woman,tfIdf.true,tfIdf.theme,tfIdf.ass,tfIdf.play,tfIdf.assault,tfIdf.period,tfIdf.medical,tfIdf.ceases,tfIdf.colors,tfIdf.america,tfIdf.disappoint,tfIdf.son,tfIdf.children,tfIdf.future,tfIdf.ground,tfIdf.charged,tfIdf.fear,tfIdf.narrative,tfIdf.interrupt,tfIdf.standard,tfIdf.london,tfIdf.evening,tfIdf.nero,...,tfIdf.federal,tfIdf.soared,tfIdf.sank,tfIdf.outchea,tfIdf.leoseason,tfIdf.tufff,tfIdf.echoing,tfIdf.chrome,tfIdf.fordtaurussho,tfIdf.rgfxcustoms,tfIdf.ford,tfIdf.delete,tfIdf.mchromedelete,tfIdf.stealthmode,tfIdf.taurus,tfIdf.glossblack,tfIdf.sho,tfIdf.blackedout,tfIdf.chromedelete,tfIdf.ingrates,tfIdf.phonys,tfI

## Export

Export in the TRECIS format

In [43]:
# write to .run file
with open("marks2.run" , "w") as out_file:
    for row in df.drop_duplicates(subset="postID").itertuples():
        #print("row:", row)
        content = [
            "TRECIS-CTIT-H-Test-0" + str(int(row.eventID)),
            "Q0",          
            np.int64(row.postID),   
            getattr(row, 'Index'),  #ToDo: Fix?
            #row.priority,
            str(priority_scorer[str(round(row.postPriority))[:3]]),  #ToDo: Fix
            row.categories,
            "marksrun2"
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")

In [ ]:
#

In [ ]:
#df['tweets'] = df['tweets'].str[2:-1]
df
